In [22]:
import torch
import numpy as np
import pyro
import pyro.distributions as dist
from pyro.infer import MCMC, NUTS

import pandas as pd
from matplotlib import pyplot as plt
import random
import time
import models
import gdown

from scipy.stats import nbinom,norm,truncnorm
from scipy.integrate import trapezoid as trapz

In [48]:
url = "https://docs.google.com/spreadsheets/d/e/2PACX-1vTltw5xQVB_sSBCLiA5nzbiDc1srkInw5TDBWcYy50A-Yhr7zKMTgJUy0aoE1q0uCo5WUJSJSVhR_SY/pub?gid=1190540286&single=true&output=csv"
df = pd.read_csv(url)

In [49]:
#file_path = 'results.ods'

# Read the Excel file
#df = pd.read_excel(file_path, engine="odf")

# Print the DataFrame
print(df)

winners=np.array(df['Winner\'s name'])
losers=np.array(df['Loser\'s name'])
loser_scores=np.array(df['Loser\'s score'])
names = pd.concat([df['Winner\'s name'], df['Loser\'s name']]).unique()
ids = list(range(len(names)))
dict_to_id = dict(zip(names,ids))
dict_to_name=dict(zip(ids,names))
winner_ids = np.array([dict_to_id[i] for i in winners])
loser_ids = np.array([dict_to_id[i] for i in losers])

    Chronological info Winner's name Loser's name  Loser's score
0                  NaN          Fedo        Mauro              1
1                  NaN          Fedo        Idris              3
2                  NaN         Mauro        Idris              6
3                  NaN          Fedo        Idris              5
4                  NaN         Mauro        Idris              8
5                  NaN          Fedo        Mauro              5
6                  NaN         Mauro        Frago              7
7                  NaN         Mauro        Frago              7
8                  NaN         Frago        Mauro              6
9                  NaN         Frago        Idris              9
10                 NaN         Frago        Idris              9
11                 NaN         Frago        Idris             10
12                 NaN         Frago        Idris              8
13                 NaN         Mauro       Satvik              7
14                 NaN   

In [30]:
winner_ids=torch.Tensor(winner_ids).int()
loser_ids=torch.Tensor(loser_ids).int()
loser_scores=torch.Tensor(loser_scores).int()

In [31]:
# Run inference using NUTS (No-U-Turn Sampler) in MCMC
nuts_kernel = NUTS(models.pyro_model)
mcmc_run = MCMC(nuts_kernel, num_samples=100, warmup_steps=10)
mcmc_run.run(winner_ids, loser_ids, loser_scores)

# Get posterior samples
posterior_samples = mcmc_run.get_samples()




Warmup:   5%|▏   | 5/110 [00:00,  9.85it/s, step size=4.57e-03, acc. prob=0.377]

KeyboardInterrupt: 

In [ ]:
# Print the posterior mean and standard deviation of player strengths
for i in range(len(ids)):
    player = f'Player {i+1}: {dict_to_name[i]}'
    print(player)
    print("Posterior mean of strength:", posterior_samples['strength'][:, i].mean().item())
    print("Posterior std dev of strength:", posterior_samples['strength'][:, i].std().item())
    print()

## Adding Gibbs sampling

In [50]:
#just in case you run this model after the previous one
if isinstance(winner_ids, torch.Tensor):
    winner_ids=winner_ids.numpy()
    loser_ids=loser_ids.numpy()
    loser_scores=loser_scores.numpy()

In [51]:
g=models.gibbs_model(winner_ids, loser_ids, loser_scores,names=names,prior_std=0.3)

In [52]:
g.posterior_sampling(n_iterations=500, warmup=100)

Progress: [################################################################################################################################################################################################################################################################################################################################################################################################################################################################################################################### ] 99%

In [53]:
g.print_table()

Player 1: Fedo
Posterior mean of strength: 0.9415634166345379
Posterior std dev of strength: 0.04151418317011727

Player 2: Mauro
Posterior mean of strength: 0.802819268155459
Posterior std dev of strength: 0.10809934303480677

Player 3: Frago
Posterior mean of strength: 0.7369261073256225
Posterior std dev of strength: 0.10896872918353052

Player 4: Idris
Posterior mean of strength: 0.502482240495777
Posterior std dev of strength: 0.06637978480888952

Player 5: Nour
Posterior mean of strength: 0.457727270427627
Posterior std dev of strength: 0.08626135970827041

Player 6: Younes
Posterior mean of strength: 0.3861880383990237
Posterior std dev of strength: 0.10776831078510857

Player 7: Andrea
Posterior mean of strength: 0.449819511951748
Posterior std dev of strength: 0.11251715054177282

Player 8: Satvik
Posterior mean of strength: 0.3842688942235295
Posterior std dev of strength: 0.09227277566527009

Player 9: André
Posterior mean of strength: 0.6074238710311392
Posterior std dev of